This notebook accompanies the Whisper Connected Fracture Analysis note.

In [55]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import LineString

In [56]:
raw = pd.read_csv(r"C:\Users\scott.mckean\Desktop\correlated_events_and_stages.csv").drop('Unnamed: 0', axis=1)
raw.head()

,Event,X,Y,Z_Actual,Mag,Well,Stage,Subsea,Stage_x_m,Stage_y_m,Stage_z_m,distance_km,Stage_start,Stage_end,time_diff_hr,datetime_recom,Diff_m2s-1
0,4039,643301.8055,6215742.323,-1222.34,-0.489,F_Well,1,725.66,643741.6998,6216163.864,-1197.705356,0.609763,1/11/2017 23:05,1/12/2017 0:43,0.241389,1/11/2017 23:19,34.048048
1,180,643299.0116,6215734.544,-1172.34,0.077,F_Well,1,725.66,643741.6998,6216163.864,-1197.705356,0.617197,1/11/2017 23:05,1/12/2017 0:43,0.384167,1/11/2017 23:28,21.918726
2,942,643298.2576,6215742.205,-1179.34,-0.348,F_Well,1,725.66,643741.6998,6216163.864,-1197.705356,0.612188,1/11/2017 23:05,1/12/2017 0:43,0.616944,1/11/2017 23:42,13.428012
3,33,643665.0895,6216357.291,-1241.34,0.819,F_Well,1,725.66,643741.6998,6216163.864,-1197.705356,0.212573,1/11/2017 23:05,1/12/2017 0:43,0.693333,1/11/2017 23:46,1.440656
4,97,643288.7731,6215736.988,-1187.34,0.386,F_Well,1,725.66,643741.6998,6216163.864,-1197.705356,0.622473,1/11/2017 23:05,1/12/2017 0:43,0.723333,1/11/2017 23:48,11.841065


In [46]:
stages = gpd.GeoDataFrame(raw, geometry=gpd.points_from_xy(x=raw['Stage_x_m'], y=raw['Stage_y_m'])).copy()
events = gpd.GeoDataFrame(raw, geometry=gpd.points_from_xy(x=raw['X'], y=raw['Y'])).copy()
events.head()

,Event,X,Y,Z_Actual,Mag,Well,Stage,Subsea,Stage_x_m,Stage_y_m,Stage_z_m,distance_km,Stage_start,Stage_end,time_diff_hr,datetime_recom,Diff_m2s-1,geometry
0,4039,643301.8055,6215742.323,-1222.34,-0.489,F_Well,1,725.66,643741.6998,6216163.864,-1197.705356,0.609763,1/11/2017 23:05,1/12/2017 0:43,0.241389,1/11/2017 23:19,34.048048,POINT (643301.806 6215742.323)
1,180,643299.0116,6215734.544,-1172.34,0.077,F_Well,1,725.66,643741.6998,6216163.864,-1197.705356,0.617197,1/11/2017 23:05,1/12/2017 0:43,0.384167,1/11/2017 23:28,21.918726,POINT (643299.012 6215734.544)
2,942,643298.2576,6215742.205,-1179.34,-0.348,F_Well,1,725.66,643741.6998,6216163.864,-1197.705356,0.612188,1/11/2017 23:05,1/12/2017 0:43,0.616944,1/11/2017 23:42,13.428012,POINT (643298.258 6215742.205)
3,33,643665.0895,6216357.291,-1241.34,0.819,F_Well,1,725.66,643741.6998,6216163.864,-1197.705356,0.212573,1/11/2017 23:05,1/12/2017 0:43,0.693333,1/11/2017 23:46,1.440656,POINT (643665.090 6216357.291)
4,97,643288.7731,6215736.988,-1187.34,0.386,F_Well,1,725.66,643741.6998,6216163.864,-1197.705356,0.622473,1/11/2017 23:05,1/12/2017 0:43,0.723333,1/11/2017 23:48,11.841065,POINT (643288.773 6215736.988)


In [62]:
connections_out = []
for event in events.Event:
    stg_geom = stages.query(f"Event == {event}").geometry.iloc[0]
    event_geom = events.query(f"Event == {event}").geometry.iloc[0]
    conn_geom = LineString([stg_geom, event_geom])
    conn_out = gpd.GeoDataFrame(
        stages.query(f"Event == {event}").reset_index(),
        geometry=[conn_geom]
        )
    connections_out.append(conn_out)

In [63]:
connections_gdf = gpd.GeoDataFrame(pd.concat(connections_out))
connections_gdf.to_file('manual_connections.shp')
connections_gdf.to_file('manual_connections.geojson', driver='GeoJSON')

c:\Users\scott.mckean\Miniconda3\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
C:\Users\scott.mckean\AppData\Local\Temp\ipykernel_4332\1920844102.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  connections_gdf.to_file('manual_connections.shp')
